In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
address = '316 Lafayette St, New York City, NY 10012'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7254535652174, -73.9951589130435.


In [7]:
#load FB access key
import yaml
FB_CONFIG_FILE = 'FB.env'
with open(FB_CONFIG_FILE, 'r') as config_file:
    config = yaml.load(config_file, Loader=yaml.BaseLoader)

# set parameters for FourSquare API calls
fb_access = config['FB']['access_token']
radius = 1000 # 1KM radius
limit = 5000 # 1K result limit

In [90]:
def fb_getNearbyVenues(coords, categoryID='', q_name='',radius=500 ):
    venues_list=[]
    for lat, lng in coords:
        #print(q_name)
            
        # create the API request URL
        url = ('https://graph.facebook.com/v3.2/search?type=place&center={},{},&distance={}'
#               '&categories={}'
               '&q={}'
               '&fields=name,checkins,location,category_list,engagement,overall_star_rating,rating_count'
               '&limit={}&access_token={}'
        ).format(
            lat, 
            lng, 
            radius,
#            categoryID,
            q_name,
            limit,
            fb_access
        )
            
        # make the GET request
#        print("URL:"+url)
        results = requests.get(url).json()["data"]
#        print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['id'],
            v['name'], 
            v['location']['latitude'], 
            v['location']['longitude'],  
#            ','.join
            list(category['name'] for category in v['category_list']),
            v['engagement']['count'],
            v['checkins']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [ 
                  'Venue ID',
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Likes',
                  'Checkins']
    
    return(nearby_venues)

In [10]:
import math
#get geo coord shift by distance
def shift_coord(lat,lon, dist,bearing):
    R = 6371.0088 #Radius of the Earth
    brng = math.radians(bearing)


    lat1 = math.radians(lat) #lat point converted to radians
    lon1 = math.radians(lon) #long point converted to radians

    lat2 = math.asin( math.sin(lat1)*math.cos(dist/R) +
    math.cos(lat1)*math.sin(dist/R)*math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(dist/R)*math.cos(lat1),
             math.cos(dist/R)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    return (lat2, lon2)
#build coord grid
def cgrid(latitude, longitude, distance, legs):
    ab=[]
    for i in range(-legs,legs):
        if i==0:
            ab.append([latitude, longitude])
            continue
        #ab.append(shift_coord(latitude, longitude, distance*i, j) for j in (0,90,180,270))
        for j in (0,90,180,270):
            ab.append(shift_coord(latitude, longitude, distance*i, j))
        #ab.append(bb)    
    return ab


In [11]:
print("Coord grid for location "+str(latitude)+','+str(longitude))
coords =cgrid(latitude, longitude, 0.25, 2) 
print(coords)


Coord grid for location 40.7254535652174,-73.9951589130435
[(40.72095696339877, -73.9951589130435), (40.72545341331191, -74.00109232208771), (40.729950167036016, -73.9951589130435), (40.72545341331191, -73.9892255039993), (40.72320526430809, -73.9951589130435), (40.72545352724103, -73.998125617569), (40.72770186612671, -73.9951589130435), (40.72545352724103, -73.992192208518), [40.7254535652174, -73.9951589130435], (40.72770186612671, -73.9951589130435), (40.72545352724103, -73.992192208518), (40.72320526430809, -73.9951589130435), (40.72545352724103, -73.998125617569)]


In [221]:
fb_cat_food='["FOOD_BEVERAGE"]'
#[(latitude, longitude)]
df_coffee = fb_getNearbyVenues([(latitude, longitude)], categoryID='',q_name='coffee shop',radius=10000)
df_coffee

,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Likes,Checkins
0,10150160592938090,Coffee Shop,40.736440,-73.990990,[American Restaurant],8092,24939
1,265382270219054,Jimmy's American Grill&Bar,40.751360,-73.986390,[Bar & Grill],211,5085
2,463044183789165,Andrew's Coffee Shop,40.751695,-73.989917,"[American Restaurant, Diner, Cafe]",3490,18283
3,1423074811282455,"The Coffee Shop, Bo. Barretto",40.750641,-74.003736,[Coffee Shop],21,392
4,115627791793313,Landmark Coffee Shop,40.719990,-73.998660,[Cafe],83,2033
5,2204712346454177,The Coffee Shop,40.726050,-73.940900,[Coffee Shop],82,15
6,861826873837278,Central Perk Café - Friends' 20th Anniversary,40.721150,-73.997836,[Coffee Shop],1319,11050
7,225014604681247,Veselka Coffee Shop,40.729620,-73.987800,[Restaurant],7,193
8,347450705899732,El Noel Coffee Shop,40.682480,-73.966530,[Coffee Shop],101,6
9,111709532198963,Westside Coffee Shop,40.720090,-74.003660,[Coffee Shop],23,432


In [182]:
#df_coffe2 = fb_getNearbyVenues([(latitude, longitude)], categoryID='',q_name='coffee',radius=3000)
#df_coffe2["Venue Category"].where(
s = df_coffe2[~df_coffe2["Venue Name"].str.contains("Starbucks")]
s

,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Likes,Checkins
0,188174614578929,Gasoline Alley Coffee,40.725420,-73.994820,"[Coffee Shop, Restaurant]",1746,3058
1,1153183334863823,Coffee 'n Clothes,40.726620,-73.994350,[Internet Cafe],1568,88
2,167155753349004,Kaffe 1668,40.715410,-74.011030,[Coffee Shop],683,6154
3,10150160592938090,Coffee Shop,40.736440,-73.990990,[American Restaurant],8092,24939
4,174894656545641,OddFellows Ice Cream Nolita,40.724580,-73.994450,"[Coffee Shop, Ice Cream Shop]",52,412
5,1488169154738884,Coffee Project New York,40.726990,-73.989220,[Coffee Shop],1779,3765
6,1783058001956701,Think Coffee,40.725420,-73.992550,"[Coffee Shop, Bakery, Tea Room]",59,783
8,191414257565217,Caffe Roma,40.720360,-73.996760,[Coffee Shop],1361,13424
9,169967663018216,Grey Dog Coffee,40.723047,-73.995806,"[Breakfast & Brunch Restaurant, Caterer]",265,5287
10,1805355666455386,Think Coffee,40.728207,-73.995365,"[Coffee Shop, Restaurant, Bakery]",83,1623


In [166]:
df_dd = fb_getNearbyVenues([(latitude, longitude)], categoryID='',q_name='dunkin',radius=3000)

In [169]:
df_sb = fb_getNearbyVenues([(latitude, longitude)], categoryID='',q_name='starbucks',radius=3000)

In [205]:
#df_coffee = df_coffee[~df_coffee["Venue Name"].str.contains("Starbucks")]
#df_coffee["Chain"]="Other"
#df_dd["Chain"]="Dunkin"
#df_sb['Chain']="Starbucks"
df_cf =pd.concat([df_coffee, df_sb] )#df_dd
df_cf


,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Likes,Checkins,Chain
0,188174614578929,Gasoline Alley Coffee,40.725420,-73.994820,"[Coffee Shop, Restaurant]",1746,3058,Other
1,1153183334863823,Coffee 'n Clothes,40.726620,-73.994350,[Internet Cafe],1568,88,Other
2,167155753349004,Kaffe 1668,40.715410,-74.011030,[Coffee Shop],683,6154,Other
3,10150160592938090,Coffee Shop,40.736440,-73.990990,[American Restaurant],8092,24939,Other
4,174894656545641,OddFellows Ice Cream Nolita,40.724580,-73.994450,"[Coffee Shop, Ice Cream Shop]",52,412,Other
5,1488169154738884,Coffee Project New York,40.726990,-73.989220,[Coffee Shop],1779,3765,Other
6,1783058001956701,Think Coffee,40.725420,-73.992550,"[Coffee Shop, Bakery, Tea Room]",59,783,Other
8,191414257565217,Caffe Roma,40.720360,-73.996760,[Coffee Shop],1361,13424,Other
9,169967663018216,Grey Dog Coffee,40.723047,-73.995806,"[Breakfast & Brunch Restaurant, Caterer]",265,5287,Other
10,1805355666455386,Think Coffee,40.728207,-73.995365,"[Coffee Shop, Restaurant, Bakery]",83,1623,Other


In [206]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=15)
chain_color = { "Starbucks":"green","Dunkin":"purple","Other":"blue"}

# add markers to map
for lat, lng, venue, checkins, chain in zip(df_cf['Venue Latitude'], df_cf['Venue Longitude'], df_cf['Venue Name'], df_cf['Checkins'], df_cf["Chain"]):
    label = venue+", Checkins:"+str(checkins)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=chain_color.get(chain),
        fill=True,
        fill_color=chain_color.get(chain),
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [86]:
#import string
s="{'name': 'Champion Pizza', 'checkins': 10834, 'location': {'city': 'New York', 'country': 'United States', 'latitude': 40.721614, 'longitude': -73.997212, 'state': 'NY', 'street': '17 Cleveland place', 'zip': '10012'}, 'category_list': [{'id': '180256082015845', 'name': 'Pizza Place'}, {'id': '192803624072087', 'name': 'Fast Food Restaurant'}], 'engagement': {'count': 524634, 'social_sentence': '524K people like this.'}, 'rating_count': 0, 'id': '184927038672914'}"
s = s.replace("'",'"')
results = json.loads(s)
print(results["category_list"])
#cc = ','.join(category['name'] for category in results['category_list'])
cc = list(category['name'] for category in results['category_list'])
cc

[{'id': '180256082015845', 'name': 'Pizza Place'}, {'id': '192803624072087', 'name': 'Fast Food Restaurant'}]


['Pizza Place', 'Fast Food Restaurant']

In [197]:
#build a KNN model (nearest neigbour)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline

In [232]:
# Define which columns should be encoded vs scaled
columns_to_encode = ['Chain']
columns_to_scale  = ['Venue Latitude','Venue Longitude','Checkins']

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe    = OneHotEncoder(sparse=False)

# Scale and Encode Separate Columns
scaled_columns  = scaler.fit_transform(df_cf[columns_to_scale]) 
encoded_columns =    ohe.fit_transform(df_cf[columns_to_encode])

# Concatenate (Column-Bind) Processed Columns Back Together
X = np.concatenate([scaled_columns, encoded_columns], axis=1)
#X = scaled_columns
#X = df_cf["Checkins"]
y = df_cf["Likes"]

C:\Users\Kris\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Kris\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [233]:
X

array([[-3.36747263e-01,  2.67572925e-01, -8.48367935e-02,
         1.00000000e+00,  0.00000000e+00],
       [-2.51647092e-01,  3.19575556e-01, -7.18637542e-01,
         1.00000000e+00,  0.00000000e+00],
       [-1.04662453e+00, -1.52596465e+00,  5.75852472e-01,
         1.00000000e+00,  0.00000000e+00],
       [ 4.44755978e-01,  6.91339051e-01,  4.58458886e+00,
         1.00000000e+00,  0.00000000e+00],
       [-3.96317383e-01,  3.08511167e-01, -6.49495642e-01,
         1.00000000e+00,  0.00000000e+00],
       [-2.25407872e-01,  8.87178750e-01,  6.60376608e-02,
         1.00000000e+00,  0.00000000e+00],
       [-3.36747263e-01,  5.18734572e-01, -5.70323899e-01,
         1.00000000e+00,  0.00000000e+00],
       [-6.95586320e-01,  5.29237637e-02,  2.12727720e+00,
         1.00000000e+00,  0.00000000e+00],
       [-5.05016851e-01,  1.58440324e-01,  3.90833869e-01,
         1.00000000e+00,  0.00000000e+00],
       [-1.39095023e-01,  2.07283065e-01, -3.91067121e-01,
         1.00000000e+00

In [234]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (137, 5) (137,)
Test set: (35, 5) (35,)


In [235]:
for k in range(1,12):  #Train Model and Predict  
  neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
  yhat = neigh.predict(X_test)
  print("For k=",k)
  print("Train set Accuracy: ", metrics.accuracy_score(y_train,neigh.predict(X_train)))
  print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))


For k= 1
Train set Accuracy:  1.0
Test set Accuracy:  0.0
For k= 2
Train set Accuracy:  0.5182481751824818
Test set Accuracy:  0.0
For k= 3
Train set Accuracy:  0.35036496350364965
Test set Accuracy:  0.0
For k= 4
Train set Accuracy:  0.23357664233576642
Test set Accuracy:  0.0
For k= 5
Train set Accuracy:  0.1897810218978102
Test set Accuracy:  0.0
For k= 6
Train set Accuracy:  0.145985401459854
Test set Accuracy:  0.0
For k= 7
Train set Accuracy:  0.10948905109489052
Test set Accuracy:  0.02857142857142857
For k= 8
Train set Accuracy:  0.0948905109489051
Test set Accuracy:  0.0
For k= 9
Train set Accuracy:  0.072992700729927
Test set Accuracy:  0.0
For k= 10
Train set Accuracy:  0.072992700729927
Test set Accuracy:  0.0
For k= 11
Train set Accuracy:  0.072992700729927
Test set Accuracy:  0.0
